In [2]:
import pickle

# Specify the path to your .p file
file_path = "/Users/tianyichen/Desktop/Research /PhDresearch/Twitters/2024-06-10-tweets-with-text.p"

# Open the file in binary mode
with open(file_path, "rb") as file:
    # Load the data from the file
    data = pickle.load(file)


In [3]:
data.head()

,name,party,twitter_lower,years,chamber,state,postedAt,source,tweetId,name_standardized,text
0,Josh Gottheimer D-NJ,D,repjoshg,2021_2022,House,NJ,2022-05-29 13:22:24.171,PolitWoops,1530902377196314624,josh_gottheimer,Thanks to the Bipartisan Infrastructure Bill t...
1,Ed Markey D-MA,D,senmarkey,2011_2012,Senate,MA,2011-05-13 21:06:26.255,PolitWoops,69146505528868864,ed_markey,"Since COPPA in 98, kids more likely 2 be poked..."
2,Debbie Stabenow D-MI,D,senstabenow,2021_2022,Senate,MI,2022-03-02 01:14:27.293,PolitWoops,1498829050919411712,debbie_stabenow,My virtual guest at the State of the Union wil...
3,Ted Cruz R-TX,R,sentedcruz,2019_2020,Senate,TX,2019-10-31 16:48:33.055,PolitWoops,1189947274526384128,ted_cruz,.@tedcruz offering rare praise for Facebook on...
4,Dan Bishop R-NC,R,repdanbishop,2021_2022,House,NC,2021-08-25 00:15:49.917,PolitWoops,1430322991376834560,dan_bishop,
